In [1]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/hubble/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/hubble/')
import pandas as pd

os.environ['PATH'] = '/Users/ravi.tej/anaconda3/envs/bertopicenv/bin:/Users/ravi.tej/anaconda3/condabin:/usr/bin:/bin:/usr/sbin:/sbin'

In [2]:
from src.SummaryService import SummaryService

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
from src.SummaryService import tokenizer

In [4]:
class TokenIterator:
    def __init__(self, stream):
        self.byte_iterator = iter(stream)
        self.buffer = io.BytesIO()
        self.read_pos = 0

    def __iter__(self):
        return self

    def __next__(self):
        while True:
            self.buffer.seek(self.read_pos)
            line = self.buffer.readline()
            if line and line[-1] == ord("\n"):
                self.read_pos += len(line) + 1
                full_line = line[:-1].decode("utf-8")
                line_data = json.loads(full_line.lstrip("data:").rstrip("/n"))
                return line_data["token"]["text"]
            chunk = next(self.byte_iterator)
            self.buffer.seek(0, io.SEEK_END)
            self.buffer.write(chunk["PayloadPart"]["Bytes"])

In [5]:
import boto3
from botocore.config import Config

In [6]:
my_config = Config(
    region_name='ap-south-1',
    retries={
        'max_attempts': 3,
        'mode': 'standard'
    },
    max_pool_connections=40  # Increase the pool size
)

# Create a SageMaker Runtime client with the custom configuration
sess1 = boto3.session.Session()
sagemaker_runtime = sess1.client("sagemaker-runtime", config=my_config)

03/05/2024 03:39:08 PM - Found credentials in shared credentials file: ~/.aws/credentials


In [7]:
import json
import io
from src.HybridSearchService import HybridSearchService

03/05/2024 03:39:09 PM - Found credentials in shared credentials file: ~/.aws/credentials


In [8]:
q = 'what is the difference between pre money and post money'

In [9]:
res = HybridSearchService.perform_hybrid_reranked_search(q, recency_importance='medium')

In [10]:
top = [x['article_id'] for x in res['searchArticleIds'][:3]]

In [11]:
top

['65367fdd1e5cc42b1b143c4d',
 '653680161e5cc42b1b143fb0',
 '6555b49e4b13023f9348b6d0']

In [12]:
prompt = SummaryService.create_prompt_request_params_summary(query = q, top_article_ids=top, token_limit=3500)

In [13]:
prompt['parameters']

{'do_sample': True,
 'top_p': 0.95,
 'temperature': 0.5,
 'max_new_tokens': 512,
 'repetition_penalty': 1.1,
 'return_full_text': False,
 'stop': ['###', '</s>', '<|im_end|>']}

In [14]:
len(tokenizer.encode(prompt['inputs']))

2849

In [15]:
response = sagemaker_runtime.invoke_endpoint_with_response_stream(
        EndpointName='OpenHermes-Search-PinkSparrowOnTV',
        Body=json.dumps(prompt),
        ContentType="application/json",
    )
for token in TokenIterator(response["Body"]):
    print(token, end="")

**Understanding Pre- and Post-Money Valuation in Startups**
💼 **Pre-Money Value**: Before investment, startup's valuation is "pre-money".
📈 **Post-Money Value**: Post-money value is calculated after the investment is injected.
💵 **Investor Investment**: The investor injects money into the startup, increasing its value.
🔄 **Valuation Increase**: The startup's value increases based on the capital infusion.
⚖️ **Capitalization Ratio**: The capitalization ratio is used to determine post-money value.
📊 **Post-Money Capitalization**: Post-money capitalization = Pre-money capitalization + capital infusion.<|im_end|>

In [15]:
len(tokenizer.encode('''**Understanding the Differences Between ULIP and Traditional Life Insurance**<br><br>**Introduction:**<br>Choosing the right financial product for your needs requires a clear understanding of what each has to offer. Two popular choices for securing your future and finances are Unit-Linked Insurance Plans (ULIPs) and traditional life insurance policies. This deep dive aims to clarify the distinctions between these two options.<br><br>**What is a ULIP?**<br>ULIPs serve dual purposes as both an insurance and investment vehicle. A portion of the premium goes towards life insurance coverage, while the rest is invested in equity, debt, or hybrid funds based on the policyholder's risk appetite and financial goals. Launched in 1971, ULIPs have evolved significantly, offering a variety of funds to match individual needs.<br><br>**What is Traditional Life Insurance?**<br>Traditional life insurance focuses solely on providing a death benefit to the nominee upon the policyholder's demise. It does not offer any investment opportunities, making it a pure protection plan. Despite this, it remains a cornerstone for many households as a safety net.<br><br>**Key Distinctions:**<br>1. **Purpose:** ULIPs aim to provide insurance and investment returns, whereas traditional life insurance focuses purely on providing a death benefit.<br>2. **Returns:** ULIPs offer potential for higher returns through market-linked investments. Traditional life insurance does not offer investment returns beyond maturity.<br>3. **Flexibility:** ULIPs allow for switching between funds based on market performance, offering more control over investment outcomes. Traditional life insurance does not provide such flexibility.<br><br>**Decision Criteria:**<br>Your choice between ULIP and traditional life insurance should depend on your financial goals, risk tolerance, and the need for insurance versus investment growth. If you seek a balance between insurance and investment, ULIP might be preferable. If your primary concern is having a safety net for your family, traditional life insurance could be more suitable.<br><br>**Conclusion:**<br>Both ULIP and traditional life insurance serve vital roles in a well-rounded financial plan. While ULIPs offer the allure of potential investment returns, traditional life insurance provides peace of mind through guaranteed protection. Understanding these differences allows individuals to select the product that aligns with their specific financial objectives and risk appetite.<|im_end|>'''))

541

In [56]:
k = '''"**Sensex Performance Snapshot: Recent Trends and Market Influences**<br><br>**Introduction:**<br><br>The Indian stock market, particularly the Sensex, has been experiencing fluctuations influenced by various domestic and international factors. Here's a quick overview of how it performed recently.<br><br>**Recent Performance:**<br><br>On January  Ö 20, 2024, the Sensex showed slight upward movement, ending with a gain of 47.85 points, or 0.07%. This came amidst a backdrop of global market movements, influenced by expectations of interest rate adjustments by central banks globally, including the Federal Reserve in the U.S.<br><br>**Influencing Factors:**<br><br>Global trends, particularly concerning central bank policies and corporate earnings, play a significant role in shaping the Sensex's performance. For instance, the anticipation of rate cuts by the Federal Reserve has been a major driver for global markets, including India's, with a notable impact on technology and financial sectors.<br><br>**Performance in Late January 2024:**<br><br>By January Ö 24, 2024, the Sensex experienced a notable recovery, ending with a substantial increase of 690 points, or 0.98%. This turnaround was supported by positive global cues and optimistic forecasts for interest rate adjustments by central banks globally, along with robust tech earnings. However, the momentum was short-lived, as by January Ö 25, 2024, the Sensex faced another downturn, closing 360 points, or 0.51% lower, largely influenced by negative performances from banking and IT giants.<br><br>**Conclusion:**<br><br>The Sensex's recent performance reflects the interconnectedness of global financial markets and the influence of macroeconomic factors. Despite occasional downturns, the market shows resilience and potential for recovery driven by positive global cues, corporate earnings, and policy decisions by central banks globally."'''

In [57]:
json.loads(k)

"**Sensex Performance Snapshot: Recent Trends and Market Influences**<br><br>**Introduction:**<br><br>The Indian stock market, particularly the Sensex, has been experiencing fluctuations influenced by various domestic and international factors. Here's a quick overview of how it performed recently.<br><br>**Recent Performance:**<br><br>On January  Ö 20, 2024, the Sensex showed slight upward movement, ending with a gain of 47.85 points, or 0.07%. This came amidst a backdrop of global market movements, influenced by expectations of interest rate adjustments by central banks globally, including the Federal Reserve in the U.S.<br><br>**Influencing Factors:**<br><br>Global trends, particularly concerning central bank policies and corporate earnings, play a significant role in shaping the Sensex's performance. For instance, the anticipation of rate cuts by the Federal Reserve has been a major driver for global markets, including India's, with a notable impact on technology and financial secto

In [41]:
k = response['Body'].read()

In [50]:
json.loads(json.loads(k)[0]['generated_text'])

"**Today's Sensex Performance: A Snapshot**<br><br>**Introduction:**<br>The Indian stock market, particularly the Sensex, has seen fluctuating trends recently, reflecting global economic conditions and domestic factors. Here's a quick look into how the Sensex performed on the last trading day.<br><br>**Sensex Performance:**<br>On the last day of trading observed, the Sensex experienced a slight increase, reaching a level of 71,731.08 by 12 pm, marking a modest rise of 47.85 points, or 0.07%.<br><br>**Market Sentiment:**<br>Despite the marginal upward movement, the broader market sentiment seems cautious, mirroring global uncertainties and mixed signals from sectors like banking and IT.<br><br>**Conclusion:**<br>While the Sensex showed some resilience, the overall market mood reflects a balance between positive and negative influences, indicating the need for vigilant monitoring of both domestic and international developments."

In [ ]:
def stream_sagemaker(rqst):
    # if 'web_sources' in rqst:
    #     web_sources = rqst.pop('web_sources')
    #     logger.info(f'web search sources are {web_sources}')
    response = sagemaker_runtime.invoke_endpoint_with_response_stream(
        EndpointName='OpenHermes-Search-HummingbirdMinnowZebra-TGI',
        Body=json.dumps(rqst),
        ContentType="application/json",
    )
    logger.log(f'sagemaker response {response}')
    for token in TokenIterator(response["Body"]):
        streamed_token = token if token != "\n" else "<br>"
        streamed_summary.append(streamed_token)
        yield streamed_token
    summary_streamed_from_llm = True